In [ ]:
# importing all the required libraries

import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy
import os

In [ ]:
#create a dynamic extraction function for csv files 

def extract_csv(file_path):
    extracted_data = pd.read_csv(file_path)
    return extracted_data

extracted_data= extract_csv(r"C:\Users\Claudio\Documents\source_crm\cust_info.csv")
print(extracted_data.head())

    cst_id     cst_key cst_firstname cst_lastname cst_marital_status cst_gndr  \
0  11000.0  AW00011000           Jon        Yang                   M        M   
1  11001.0  AW00011001        Eugene      Huang                    S        M   
2  11002.0  AW00011002         Ruben       Torres                  M        M   
3  11003.0  AW00011003       Christy          Zhu                  S        F   
4  11004.0  AW00011004     Elizabeth      Johnson                  S        F   

  cst_create_date  
0      2025-10-06  
1      2025-10-06  
2      2025-10-06  
3      2025-10-06  
4      2025-10-06  


In [ ]:

#check for drivers name 

pyodbc.drivers()

#create connection engine to database

from sqlalchemy import create_engine

connection_uri = (f"mssql+pyodbc://@DESKTOP-BP1GKPI/Datawarehouse?trusted_connection=yes&driver=ODBC Driver 17 for SQL Server")
db_engine = sqlalchemy.create_engine(connection_uri)

#load the data to sql server

extracted_data.to_sql(
    name = "crm_customer_info",
    schema="bronze",
    con= db_engine,
    index=False,
    if_exists="replace"
)


255

In [ ]:
#check the count of rows and columns againist the sql server

Validation = pd.read_sql("SELECT * FROM bronze.crm_customer_info;", db_engine)
Validation.shape

(18494, 7)

In [ ]:
# extract all the required csv files manually and load them to the sql server database

extracted_data_prd = extract_csv(r"C:\Users\Claudio\Documents\source_crm\prd_info.csv")

#LOAD

extracted_data_prd.to_sql(
    name = "crm_prduct_info",
    schema="bronze",
    con= db_engine,
    index=False,
    if_exists="replace"
)
#check the count of rows and columns againist the sql server

Validation = pd.read_sql("SELECT * FROM bronze.crm_prduct_info;", db_engine)
Validation.shape



(397, 7)

In [ ]:
extracted_data_sales = extract_csv(r"C:\Users\Claudio\Documents\source_crm\sales_details.csv")

#LOAD

extracted_data_sales.to_sql(
    name = "crm_sales_details",
    schema = "bronze",
    con = db_engine,
    index = False,
    if_exists = "replace"
)
#check the count of rows and columns againist the sql server

Validation = pd.read_sql("SELECT * FROM bronze.crm_sales_details;", db_engine)
Validation.shape

(60398, 9)

In [ ]:
# Building up Dynamic load function to extract the rest of csv files automaticlly 

def load_to_sql (extracted_data, table_name, db_engine):
    extracted_data.to_sql(
        name = table_name,
        schema = "bronze",
        con = db_engine,
        index = False,
        if_exists = "replace"
    )

#create csv file to sql tables maping

file_to_table_mapping = {"CUST_AZ12": "erp_cust_az12",
                          "LOC_A101": "erp_loc_a101",
                          "PX_CAT_G1V2": "erp_px_cat_g1v2"}

#create the run_etl fuction

def run_etl(source_folder, db_engine):
    for file in os.listdir(source_folder):
        if file.endswith(".csv"):
            file_name_without_ext = os.path.splitext(file)[0]
            table_name = file_to_table_mapping.get(file_name_without_ext)

        if table_name:
            file_path = os.path.join(source_folder, file)
            extracted_data = extract_csv(file_path)
            load_to_sql (extracted_data, table_name,db_engine)

source_folder = r"C:\Users\Claudio\Documents\source_erp"
run_etl(source_folder, db_engine)

In [ ]:
#check the count of rows and columns againist the sql server

Validation = pd.read_sql("SELECT * FROM bronze.erp_cust_az12;", db_engine)
Validation.shape

(18484, 3)

In [56]:
Validation = pd.read_sql("SELECT * FROM bronze.erp_loc_a101", db_engine)
Validation.shape

(18484, 2)

In [58]:
Validation = pd.read_sql("SELECT * FROM bronze.erp_px_cat_g1v2;", db_engine)
Validation.shape

(37, 4)